In [1]:
from main import  SoilingLossForecast
from configs.config import TrainTestPipelineConfig, LoggerConfig
from configs import log
import logging

2023-07-09 00:08:31.272093: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 00:08:31.395621: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 00:08:31.418021: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-09 00:08:31.769478: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
import numpy as np
import pandas as pd


In [3]:
config_list = ["./pipeline_config_TFT.yaml"]
logger_config_list = ["./pipeline_config_TFT.yaml"]
logger_config = LoggerConfig(logger_config_list)
log.setup_logging(logger_config)
logger = logging.getLogger(__name__)
pipeline_config = TrainTestPipelineConfig(config_list)
forecast_obj = SoilingLossForecast(pipeline_config)
forecast_obj.read_data()

2023-07-09 00:08:38 configs.config                                - DEBUG    — _get_config:62 - Loading 'TrainTestPipelineConfig' config from '/home/sai/Pictures/TFT_fusion/pipeline_config_TFT.yaml'


/home/sai/Pictures/TFT_fusion/datapreprocessor/dataloader.py:54: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=1)
/home/sai/Pictures/TFT_fusion/datapreprocessor/dataloader.py:54: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=1)
/home/sai/Pictures/TFT_fusion/datapreprocessor/dataloader.py:140: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
/home/sai/Pictures/TFT_fusion/datapreprocessor/dataloader.py:140: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])


In [31]:
data = forecast_obj.data.copy()

In [32]:
len(data)

3143572

In [33]:
data.isnull().sum()

TIMESTAMP      4
RECORD         4
GeffRef        4
GeffTest       4
IscRef        69
IscTest       69
TempRef        4
TempTest       4
year_month     0
location       0
dtype: int64

In [35]:
data.dropna( how="any", inplace=True)

In [36]:
data.isnull().sum()

TIMESTAMP     0
RECORD        0
GeffRef       0
GeffTest      0
IscRef        0
IscTest       0
TempRef       0
TempTest      0
year_month    0
location      0
dtype: int64

In [42]:
karnal = data.loc[data["location"]=="P-1_Karnal"]

In [43]:
karnal

,TIMESTAMP,RECORD,GeffRef,GeffTest,IscRef,IscTest,TempRef,TempTest,year_month,location
2112986,2022-07-30 00:01:00,167862,0.0108511,0.02538246,8.043885E-05,0.0001876906,24.84,24.43,Aug22,P-1_Karnal
2112987,2022-07-30 00:01:30,167863,0.01083818,-0.009661586,8.043885E-05,-7.15012E-05,25.04,24.56,Aug22,P-1_Karnal
2112988,2022-07-30 00:02:00,167864,-0.01083198,-0.009665909,-8.043885E-05,-7.15012E-05,25.14,24.49,Aug22,P-1_Karnal
2112989,2022-07-30 00:02:30,167865,-0.0289536,0.01086917,-0.0002145036,8.043885E-05,24.74,24.56,Aug22,P-1_Karnal
2112990,2022-07-30 00:03:00,167866,0.01204698,0.01448372,8.93765E-05,0.0001072518,24.98,24.66,Aug22,P-1_Karnal
...,...,...,...,...,...,...,...,...,...,...
3143567,2022-12-07 17:57:30,544415,0.3669072,0.4718435,0.002654482,0.003396307,20.73,19.86,Dec22,P-1_Karnal
3143568,2022-12-07 17:58:00,544416,0.3335149,0.3961517,0.002422103,0.002860048,21.38,20.37,Dec22,P-1_Karnal
3143569,2022-12-07 17:58:30,544417,0.3100407,0.3367991,0.00224335,0.002431041,20.75,20.34,Dec22,P-1_Karnal
3143570,2022-12-07 17:59:00,544418,0.249164,0.2922041,0.001805405,0.002118223,20.99,21.07,Dec22,P-1_Karnal


In [45]:
karnal_30s = karnal.asfreq("30S")

In [ ]:
karnal_30s.resample("5M")

In [11]:
data.reset_index(inplace=True)

In [38]:
stack_data = data.copy()

In [39]:
 # Convert 'Timestamp' column to datetime
stack_data['TIMESTAMP'] = pd.to_datetime(stack_data['TIMESTAMP'])

# Set 'Timestamp' column as the index
stack_data.set_index(['TIMESTAMP','location'], inplace=True)

In [40]:
stack_data

,,RECORD,GeffRef,GeffTest,IscRef,IscTest,TempRef,TempTest,year_month
TIMESTAMP,location,,,,,,,,
2022-08-07 00:01:00,P-3_Hisar,173916,0.02205942,-0.01227369,0.0001609563,-8.942019E-05,22.18,21.92,Aug22
2022-08-07 00:01:30,P-3_Hisar,173917,-0.01101865,-0.02822736,-8.047817E-05,-0.0002056664,22.35,21.94,Aug22
2022-08-07 00:02:00,P-3_Hisar,173918,0.0122566,-0.02082119,8.942019E-05,-0.0001520143,22.16,22.28,Aug22
2022-08-07 00:02:30,P-3_Hisar,173919,0.01343591,-0.006136173,9.836221E-05,-4.47101E-05,22.74,21.94,Aug22
2022-08-07 00:03:00,P-3_Hisar,173920,-0.007341333,-0.006105782,-5.365211E-05,-4.47101E-05,22.45,22.78,Aug22
...,...,...,...,...,...,...,...,...,...
2022-12-07 17:57:30,P-1_Karnal,544415,0.3669072,0.4718435,0.002654482,0.003396307,20.73,19.86,Dec22
2022-12-07 17:58:00,P-1_Karnal,544416,0.3335149,0.3961517,0.002422103,0.002860048,21.38,20.37,Dec22
2022-12-07 17:58:30,P-1_Karnal,544417,0.3100407,0.3367991,0.00224335,0.002431041,20.75,20.34,Dec22


In [41]:
stack_data.unstack()


ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])



# Set the frequency to daily and fill missing values with 0
df = df.asfreq('D', fill_value=0)

In [16]:
import pandas as pd

# Create a sample DataFrame with multilevel index
data1 = {'Date': ['2023-07-01', '2023-07-02', '2023-07-03', '2023-07-01', '2023-07-02', '2023-07-03'],
        'Category': ['A', 'A', 'A', 'B', 'B', 'B'],
        'Value': [10, 20, 15, 5, 10, 12],
          'Value2': [100, 200, 150, 50, 100, 120],   }
df = pd.DataFrame(data1)

# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Set 'Date' and 'Category' columns as the index
df.set_index(['Date', 'Category'], inplace=True)

# Set the frequency to daily and fill missing values with 0
# df = df.asfreq('D', fill_value=0)

print(df)


                     Value  Value2
Date       Category               
2023-07-01 A            10     100
2023-07-02 A            20     200
2023-07-03 A            15     150
2023-07-01 B             5      50
2023-07-02 B            10     100
2023-07-03 B            12     120


In [18]:
df_unstack = df.unstack()

In [21]:
df_freq = df_unstack.asfreq("d")

In [22]:
df_freq.stack()

Value  Value2
Date       Category               
2023-07-01 A            10     100
           B             5      50
2023-07-02 A            20     200
           B            10     100
2023-07-03 A            15     150
           B            12     120

In [10]:
print(df.loc["2023-07-01"])

          Value
Category       
A            10
B             5


In [13]:
df = df.unstack().asfreq('d').stack()
print (df)

                     Value
Date       Category       
2023-07-01 A            10
           B             5
2023-07-02 A            20
           B            10
2023-07-03 A            15
           B            12


In [ ]:
df